In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import pickle
import csv
from streamz import Stream
from streamz.dataframe import Random
from kafka import *
from random import randint
import datetime, time, random, threading
import sys
from xlrd import open_workbook
%matplotlib inline

In [93]:
colnames = ['Time', 'r.ankle Acceleration X (m/s^2)',
	'r.ankle Acceleration Y (m/s^2)', 'r.ankle Acceleration Z (m/s^2)',
	'r.ankle Angular Velocity X (rad/s)',
	'r.ankle Angular Velocity Y (rad/s)',
	'r.ankle Angular Velocity Z (rad/s)', 'r.ankle Magnetic Field X (uT)',
	'r.ankle Magnetic Field Y (uT)', 'r.ankle Magnetic Field Z (uT)',
	'l.ankle Acceleration X (m/s^2)', 'l.ankle Acceleration Y (m/s^2)',
	'l.ankle Acceleration Z (m/s^2)', 'l.ankle Angular Velocity X (rad/s)',
	'l.ankle Angular Velocity Y (rad/s)',
	'l.ankle Angular Velocity Z (rad/s)', 'l.ankle Magnetic Field X (uT)',
	'l.ankle Magnetic Field Y (uT)', 'l.ankle Magnetic Field Z (uT)',
	'r.thigh Acceleration X (m/s^2)', 'r.thigh Acceleration Y (m/s^2)',
	'r.thigh Acceleration Z (m/s^2)', 'r.thigh Angular Velocity X (rad/s)',
	'r.thigh Angular Velocity Y (rad/s)',
	'r.thigh Angular Velocity Z (rad/s)', 'r.thigh Magnetic Field X (uT)',
	'r.thigh Magnetic Field Y (uT)', 'r.thigh Magnetic Field Z (uT)',
	'l.thigh Acceleration X (m/s^2)', 'l.thigh Acceleration Y (m/s^2)',
	'l.thigh Acceleration Z (m/s^2)', 'l.thigh Angular Velocity X (rad/s)',
	'l.thigh Angular Velocity Y (rad/s)',
	'l.thigh Angular Velocity Z (rad/s)', 'l.thigh Magnetic Field X (uT)',
	'l.thigh Magnetic Field Y (uT)', 'l.thigh Magnetic Field Z (uT)',
	'head Acceleration X (m/s^2)', 'head Acceleration Y (m/s^2)',
	'head Acceleration Z (m/s^2)', 'head Angular Velocity X (rad/s)',
	'head Angular Velocity Y (rad/s)', 'head Angular Velocity Z (rad/s)',
	'head Magnetic Field X (uT)', 'head Magnetic Field Y (uT)',
	'head Magnetic Field Z (uT)', 'sternum Acceleration X (m/s^2)',
	'sternum Acceleration Y (m/s^2)', 'sternum Acceleration Z (m/s^2)',
	'sternum Angular Velocity X (rad/s)',
	'sternum Angular Velocity Y (rad/s)',
	'sternum Angular Velocity Z (rad/s)', 'sternum Magnetic Field X (uT)',
	'sternum Magnetic Field Y (uT)', 'sternum Magnetic Field Z (uT)',
	'waist Acceleration X (m/s^2)', 'waist Acceleration Y (m/s^2)',
	'waist Acceleration Z (m/s^2)', 'waist Angular Velocity X (rad/s)',
	'waist Angular Velocity Y (rad/s)', 'waist Angular Velocity Z (rad/s)',
	'waist Magnetic Field X (uT)', 'waist Magnetic Field Y (uT)',
	'waist Magnetic Field Z (uT)', 'FileName', 'subject', 'trial_type']

meta_acceletarion = ['r.ankle Acceleration X (m/s^2)',
	'r.ankle Acceleration Y (m/s^2)', 'r.ankle Acceleration Z (m/s^2)',
    'l.ankle Acceleration X (m/s^2)', 'l.ankle Acceleration Y (m/s^2)',
	'l.ankle Acceleration Z (m/s^2)','r.thigh Acceleration X (m/s^2)', 
    'r.thigh Acceleration Y (m/s^2)','r.thigh Acceleration Z (m/s^2)',
    'l.thigh Acceleration X (m/s^2)', 'l.thigh Acceleration Y (m/s^2)',
	'l.thigh Acceleration Z (m/s^2)','head Acceleration X (m/s^2)', 
    'head Acceleration Y (m/s^2)','head Acceleration Z (m/s^2)',
    'sternum Acceleration X (m/s^2)','sternum Acceleration Y (m/s^2)', 
    'sternum Acceleration Z (m/s^2)','waist Acceleration X (m/s^2)', 
    'waist Acceleration Y (m/s^2)','waist Acceleration Z (m/s^2)']

In [76]:
df = pickle.load(open("../../full_dataset.p", "rb"))

In [52]:
# chunksize = 1
# chunks = pd.read_csv('test', chunksize=chunksize,names=columns)
# print(chunks.get_chunk(1920))

# source = Stream.from_textfile('./test2')
# example = [[{'x': 1, 'y': 30}]]
# lines = source.partition(2).to_batch(example=example) # batches of 100 elements
# records = lines.map(csv.reader)
# print(records)
# print(records.to_dataframe())
# sdf = records.to_dataframe()
# print(sdf)
# for i in range(5) :
#     source.emit(i)
# source = Stream.filenames('./test')  # stream of filenames
# sdf = (source.map(pd.read_csv)                  # stream of Pandas dataframes
#              .to_dataframe(example=df))        # logical streaming dataframe

# sdf.stream.sink(print)    

In [103]:
import time
from optparse import OptionParser

SLEEP_INTERVAL = 0.0008

def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

def readlines_then_tail(fin):
    "Iterate through lines and then tail for further lines."
    while True:
        line = fin.readline()
        if line:
            yield line
        else:
            tail(fin)

def tail(fin):
    "Listen for new lines added to file."
    while True:
        where = fin.tell()
        line = fin.readline()
        if not line:
            time.sleep(SLEEP_INTERVAL)
            fin.seek(where)
        else:
            yield line
    
def transformRow(row) :
    i = 0
    newRow = []
    for i in range(0,len(row)) :
        if isfloat(row[i]) :
            newRow.append(float(row[i]))
        else :
            newRow.append(row[i])
    return newRow

def updateTime(rows) : 
    return rows[-1][0] - rows[-2][0]
    

# Get data ready to be used in the ML model
def preprocessData(df) :
    # extraction of the trial number from the file name
    # Not Ordinal: necessary dummy variables
    df['trial_num'] = df['FileName'].apply(lambda x: x.replace('.xlsx','').split('_')[-1])

    # extraction of the trial subtype (slip, trip, pick object from ground, etc) from the file name
    # Necessary dummy variables
    df['trial_subtype'] = df['FileName'].apply(lambda x: x.split('_')[1])

    # column with the real date and time of the measurement
    # according to the README.txt, the Time column is the number of microseconds from 01/01/1970
    initial = datetime.datetime(1970,1,1,0,0,0)
    df['time_datetime'] = df.Time.apply(lambda x: (initial + datetime.timedelta(microseconds=x)))
    # creating the column 'Time_datetime' in a pandas datetime format
    df['time_datetime'] = df.time_datetime.apply(lambda t:
                            pd.datetime(t.year,t.month,t.day,t.hour,t.minute,t.second,t.microsecond))
    ###### The first 3 characters of the file name seems to be a supercategory of the trial_subtype.
    ###### This way, I believe we can discard it and use just trial_subtype
    df['target'] = np.where(df['trial_type'] == 'Falls', 1, 0)
    
def generateTimeWindows(df) :
    # group in intervals of 2 seconds, calculating the mean
    df_2s_window_mean = df.groupby(
        ['subject','trial_type','trial_subtype','trial_num',
         pd.Grouper(key='time_datetime', freq='2500000us')])[meta_acceletarion].mean()
    df_2s_window_mean = df_2s_window_mean.reset_index()
    return df_2s_window_mean

    
def main():
#     sdf = pd.DataFrame(columns=colnames)
#     with open('./test', 'r') as fin:
#         for line in readlines_then_tail(fin):
#             row = line.strip()
#             row = row.split(',')
#             transformRow(row)              
#             sdf = sdf.append(pd.DataFrame([row],columns=sdf.columns))
#             print(sdf['Time'])
    sdf = pd.DataFrame(columns=colnames)
    rowsProcessed = 0
    timePassed = 0
    with open('./streamdata.csv', 'r') as fin:
        rows = []
        for line in fin:
            rows.append([])
            rows[-1] = line.strip()
            rows[-1] = rows[-1].split(',')
            rows[-1] = transformRow(rows[-1]) 
            if (rowsProcessed > 0) :
                timePassed += updateTime(rows)
            rowsProcessed += 1
            if timePassed >= 2500000 : # After 2.5 seconds have passed, create a dataframe
                sdf = sdf.append(pd.DataFrame(rows,columns=sdf.columns))
                preprocessData(sdf)
                sdf2Sec = generateTimeWindows(sdf)
                print(sdf2Sec['r.ankle Acceleration X (m/s^2)'])
                # Reinitialize everything
                timePassed = 0
                rowsProcessed = 0
                rows = []
                del sdf
                sdf = pd.DataFrame(columns=colnames)
                
#             if (rowsProcessed * 0.0008 >= 2.5) :
#                 rowsProcessed = 0
#                 print("interval reached")


# 0.000781 seconds between each sensor row
if __name__ == '__main__':
#     print("hola")
    main()

0   -9.748290
1   -9.758122
Name: r.ankle Acceleration X (m/s^2), dtype: float64
0    -9.752942
1   -10.195902
Name: r.ankle Acceleration X (m/s^2), dtype: float64
0    -9.551224
1   -11.879573
Name: r.ankle Acceleration X (m/s^2), dtype: float64
0   -11.090764
1   -10.825161
Name: r.ankle Acceleration X (m/s^2), dtype: float64
0   -9.640672
1   -9.755768
Name: r.ankle Acceleration X (m/s^2), dtype: float64
